<a href="https://colab.research.google.com/github/dsandeep0138/HackerEarth-Challenges/blob/master/flower_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsandeep97","key":"91ed93d9f57d24d723707d120074f706"}'}

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
 !chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d rednivrug/flower-recognition-he

 99% 897M/905M [00:18<00:00, 70.0MB/s]
100% 905M/905M [00:18<00:00, 50.8MB/s]


In [0]:
!unzip flower-recognition-he.zip

In [1]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, BatchNormalization, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
traindf = pd.read_csv('data/train.csv', dtype=str)
testdf = pd.read_csv('data/test.csv', dtype=str)

In [0]:
traindf['image_id'] = traindf['image_id'] + ".jpg"
testdf['image_id'] = testdf['image_id'] + ".jpg"

In [4]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=10,
                             width_shift_range=0.1, 
                             height_shift_range=0.1)

train_generator = datagen.flow_from_dataframe(
dataframe=traindf,
directory="./data/train/",
x_col="image_id",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224, 224))

Found 18540 validated image filenames belonging to 102 classes.


In [5]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./data/test/",
x_col="image_id",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224, 224))

Found 2009 validated image filenames.


In [6]:
vgg_model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

for layer in vgg_model.layers[:]:
    layer.trainable=False
    
input = Input(shape=(224, 224, 3),name = 'image_input')
output_vgg16_conv = vgg_model(input)

x = BatchNormalization()(output_vgg16_conv)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(102, activation='softmax')(x)

model = Model(input=input, output=x)

model.summary()








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 512)         2048      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 512)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 4, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
___

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("de...)`


In [7]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=50,
                    verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
579/579 [==============================] - 218s 376ms/step - loss: 2.1970 - acc: 0.4774
Epoch 2/50
579/579 [==============================] - 217s 374ms/step - loss: 0.9779 - acc: 0.7377
Epoch 3/50
579/579 [==============================] - 216s 373ms/step - loss: 0.6318 - acc: 0.8276
Epoch 4/50
579/579 [==============================] - 215s 371ms/step - loss: 0.4466 - acc: 0.8793
Epoch 5/50
579/579 [==============================] - 217s 375ms/step - loss: 0.3410 - acc: 0.9043
Epoch 6/50
579/579 [==============================] - 215s 372ms/step - loss: 0.2659 - acc: 0.9251
Epoch 7/50
579/579 [==============================] - 215s 372ms/step - loss: 0.2160 - acc: 0.9385
Epoch 8/50
579/579 [==============================] - 217s 376ms/step - loss: 0.1806 - acc: 0.9505
Epoch 9/50
579/579 [==============================] - 215s 371ms/step - loss: 0.1590 - acc: 0.9536
Epoch 10/50
579

In [9]:
test_generator.reset()
pred = model.predict_generator(test_generator,
                               steps=STEP_SIZE_TEST,
                               verbose=1)

2009/2009 [==============================] - 28s 14ms/step


In [0]:
predicted_class_indices=np.argmax(pred, axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filenames=test_generator.filenames
filenames=[f.split('.')[0] for f in filenames]

results=pd.DataFrame({"image_id":filenames,
                      "category":predictions})
results = results.sort_values(by = ['image_id'], ascending = [True])
results.to_csv("results.csv", index=False)